<a href="https://colab.research.google.com/github/LucaswasTaken/Acessibilidade_Parte_1_Analise_de_Sentimentos/blob/main/clusterization/Clusteriza%C3%A7%C3%A3o_Artificial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pytools setup

In [ ]:
#@title Selecione o usário: { form-width: "50%" }
WORKSPACE = "HT-BZS" #@param ["HT-Annoy", "HT-BZS", "development", "HT-BZS-JOAB", "HT-ASL"] {allow-input: true}
LANGUAGE_ID = 'ase' if WORKSPACE != 'HT-BZS' else 'bzs'
user = "lucas_1" #@param ['lucas_1', 'lucas_2', 'ailton_1', 'ailton_2', 'lucas_cpp_1', 'lucas_cpp_2', 'lucas_cpp_3', 'lucas_cpp_4', 'denny_1', 'denny_2', 'wesley_1', 'wesley_2'] {allow-input: true}

from copy import deepcopy
from google.colab import auth,drive
import os
import sys
import subprocess
import json
# Montar drive

print('MONTAR DRIVE: ')
drive.mount('/content/drive')

# Autenticação é necessária para clonar

print('AUTENTICAR: ')
auth.authenticate_user()

In [ ]:
%%capture
!gcloud source repos clone github_hand-talk_pytools --project='ht-community' && pip install -r /content/github_hand-talk_pytools/requirements.txt
sys.path.insert(0, '/content/github_hand-talk_pytools')

In [ ]:
from handtalk.community import Community
import handtalk.community as community
import cereja as cj
cmt = Community(WORKSPACE)

# recuperando arquivos
segments_to_extract = cj.FileIO.load(f'/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/PROCESSOS/Clusters_Artificiais/segments_to_extract_{WORKSPACE}.json').data[user]
dict_of_prymary_segments = cj.FileIO.load(f'/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/PROCESSOS/Clusters_Artificiais/dict_of_prymary_segments_{WORKSPACE.split("-")[1]}.json').data
all_segment_sentences = cj.FileIO.load(f'/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/PROCESSOS/Clusters_Artificiais/all_segment_sentences_{WORKSPACE.split("-")[1]}.json').data
list_of_prymary_segments = cj.FileIO.load(f'/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/PROCESSOS/Clusters_Artificiais/list_of_prymary_segments_{WORKSPACE.split("-")[1]}.txt').data
if WORKSPACE == 'HT-BZS':
    definitive_sentences = cj.FileIO.load(f'/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/PROCESSOS/Clusters_Artificiais/definitive_sentences_{WORKSPACE.split("-")[1]}.json').data
try:
    next_index = int(cj.FileIO.load(f'/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/PROCESSOS/Clusters_Artificiais/results/testes/{WORKSPACE}/progress_{user}.txt').data[0])
except:
    next_index = 0

try:
    dict_signs_definite = cj.FileIO.load(f'/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/PROCESSOS/Clusters_Artificiais/results/testes/{WORKSPACE}/cluster_data_{WORKSPACE.split("-")[1]}_{user}.json').data
except:
    dict_signs_definite = cj.FileIO.load(f'/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/PROCESSOS/Clusters_Artificiais/dict_signs_definite_{WORKSPACE.split("-")[1]}.json').data

# Resgatando dados do drive

# Keypoints HT-BZS

In [3]:
#carregando dados de Keypoints a serem usados
%%capture
!cp "/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/MODELOS/SIAMESE/DATA/segments_keypoints_HT-BZS.zip" ./
!unzip /content/segments_keypoints_HT-BZS.zip

# Keypoins HT-ASL

In [ ]:
#carregando dados de Keypoints a serem usados
# %%capture
# !cp "/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/MODELOS/SIAMESE/DATA/segments_keypoints_HT-ASL.zip" ./
# !unzip /content/segments_keypoints_HT-ASL.zip

# CARREGAR Rede siamesa

In [4]:
# Funçãão de Loss utilizada nas novas redes siamesas
def contrastive_loss(y_true, y_pred):
    y_true=tf.dtypes.cast(y_true, tf.float64)
    y_pred=tf.dtypes.cast(y_pred, tf.float64)
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

In [5]:
# carregando o modelo de rede siamesa. Atualmente utilizamos o "model_siamese_test_lstm_v2.h5"
import tensorflow as tf
#caminho para pasta dos modelos
SEGMENTS_PATH = cj.Path('/content/content/segments_keypoints')
MODEL = tf.keras.models.load_model('/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/MODELOS/CLASSIFICADOR_2_JOAB/model_siamese_test_new_rescale_fn_lstm_v2.5.h5',custom_objects={'contrastive_loss':contrastive_loss})
print("Selecionado : model_siamese_test_new_rescale_fn_lstm_v2")

Selecionado : model_siamese_test_new_rescale_fn_lstm_v2


## Declaraçãão das funçõões uteis a serem utilizadas

In [6]:
# Essa éé uma funçãão especial, dados duas listas de segmentos pareados (Ids), ela fala o quãão similar cada par éé
def siamese_predict_batch(a, b):
    X1 = []
    X2 = []
    classes = []
    for seg_id in a:
        kpts = get_seg_kpts(seg_id)
        if len(kpts) == 0:
            continue
        X1.append(kpts)
    for seg_id in b:
        kpts = get_seg_kpts(seg_id)
        if len(kpts) == 0:
            continue
        X2.append(kpts)
    X1 = np.array(X1)
    X2 = np.array(X2)
    
    result = MODEL.predict([X1, X2])
    return result.reshape(len(result))

PRIMARY_SEGMENTS = None
LAST_MIN_ON_CLUSTER = 5
COUNT_SEGMENTS_ADDED = 0

#PRINCIPAL FUNÇÃÃO D ETODAS, VARRE A PASTA DE KEYPIONTS E CARREGA O KEYPOINT DO SEGMENTO DE INTERESSE
def get_seg_kpts(seg_id, seq_len=15):
    global COUNT_SEGMENTS_ADDED
    assert SEGMENTS_PATH.exists, 'Necessário baixar todos os segmentos dos primários'
    seg_file = SEGMENTS_PATH.join(f'{seg_id}.npy')
    kpts = None
    if seg_file.exists:
        try:
            kpts = np.load(seg_file)
        except Exception as err:
            print(f"Não foi possível resgatar keypoints do segmento {seg_id}: {err}")
            return []
    else:
        try:
            kpts = cmt.get_document('segments', seg_id).keypoints
            np.save(seg_file, kpts)
            COUNT_SEGMENTS_ADDED += 1
        except Exception as err:
            print(f"Não foi possível resgatar keypoints do segmento {seg_id}: {err}")
    if kpts is None or len(kpts) == 0:
        return []
    return np.array(cj.rescale_values(kpts, seq_len, interpolation=True))

# Rodando algoritmo

In [7]:
#Função de similaridade entre sentenças
import nltk
nltk.download('rslp')
from nltk.stem import RSLPStemmer
stemmer = RSLPStemmer()
def commom_words(sentence_1, sentence_2):
    if WORKSPACE == 'HT-BZS':
        sentence_1 = set([stemmer.stem(word) for word in sentence_1.split()])
        sentence_2 = set([stemmer.stem(word) if word!='' else '' for word in sentence_2])
    else:
        sentence_1 = set([stemmer.stem(word) for word in sentence_1.split()])
        sentence_2 = set([stemmer.stem(word) for word in sentence_2.split()])
    return not sentence_1.isdisjoint(sentence_2)

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


In [9]:
dict_signs_definite['no_cluster'] = []

In [ ]:
from cereja import Progress
import numpy as np
list_of_prymary_segments = np.array(list_of_prymary_segments)
count = 0
for segment in Progress.prog(segments_to_extract):

    # Calculando similaridade do segmento com os primários
    list_segment = []
    for i in range(len(list_of_prymary_segments)):
        list_segment.append(segment)
    list_segment = np.array(list_segment)
    results = siamese_predict_batch(list_of_prymary_segments,list_segment)

    #montando clusteres com as palavras relacionadas
    chosen_arg = None
    if WORKSPACE == 'HT-BZS':
        for arg in np.argsort(results):
            if results[arg]>= 0.3:
                break
            if commom_words(all_segment_sentences[segment],definitive_sentences[list_of_prymary_segments[arg]]):
                chosen_arg = arg
                break
    else:
        for arg in np.argsort(results):
            if results[arg]>= 0.15:
                break
            if commom_words(all_segment_sentences[segment],all_segment_sentences[list_of_prymary_segments[arg]]):
                chosen_arg = arg
                break
    # caso tenha enconteado palavras relacionadas com similaridade suficiente
    if chosen_arg != None:
        if results[chosen_arg] < 0.3:
            closer_primary = list_of_prymary_segments[chosen_arg]
            primary_sign = dict_of_prymary_segments[closer_primary]
            dict_signs_definite[primary_sign].append(segment)
        else:
            dict_signs_definite['no_cluster'].append(segment)
    else:
        dict_signs_definite['no_cluster'].append(segment)
    # salvando resultados
    count+=1
    if count%21 == 0:
        cj.FileIO.create(f'/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/PROCESSOS/Clusters_Artificiais/results/testes/{WORKSPACE}/progress_{user}.txt',[count]).save(exist_ok=True)
        cj.FileIO.create(f'/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/PROCESSOS/Clusters_Artificiais/results/testes/{WORKSPACE}/cluster_data_{WORKSPACE.split("-")[1]}_{user}.json',dict_signs_definite).save(exist_ok=True)

🍒 Progress » 00026/36189 - [▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱] - 0.07% - 🕜 00:13:08/16:39:49 estimated 